# Split Scheduler

In [ ]:
# Preferences:
# LEGS trained alone - OK
# LEGS trained after rest day - OK
# REST between muscle groups 3-4 - OK 
# ROTATIONS per muscle group = N - OK 
# RECALLS per muscle group = M - TODO
# REST days = K - OK
# CONSECUTIVE number of workouts - OK
# CONSECUTIVE number of rest days - OK
# MINIMIZE muscle groups per session - OK


## Muscle Group Data Structure

In [924]:
class MuscleGroup:
    def __init__(self, name, rest_min=4, rest_max=5, rotations=3, after_rest=False):
        self.name = name
        self.rest_min = rest_min
        self.rest_max = rest_max
        self.rotations = rotations
        self.after_rest = after_rest
        self.split_preference = set([])
    
    def set_split_preference(self, preference):
        self.split_preference = set(preference)


## Global Preferences

In [925]:
chest = MuscleGroup('CHEST', rest_min=3, rest_max=4, rotations=3, after_rest=False)
back = MuscleGroup('BACK', rest_min=3, rest_max=10, rotations=3, after_rest=False)
legs = MuscleGroup('LEGS', rest_min=4, rest_max=10, rotations=2, after_rest=True)
arms = MuscleGroup('ARMS', rest_min=1, rest_max=10, rotations=3, after_rest=False)
delts = MuscleGroup('DELTS', rest_min=1, rest_max=10, rotations=3, after_rest=False)

chest.set_split_preference([arms, delts])
back.set_split_preference([arms, delts])
legs.set_split_preference([])
arms.set_split_preference([chest, back, delts])
delts.set_split_preference([chest, back, arms])

groups = set([chest, back, legs, arms, delts])
days = range(14) # days of the microcycle
rest_days = 5 # number of rest days of the microcycle
max_consecutive_work = 3 # number of max consecutive days of training
max_consecutive_rest = 2 # number of max consecutive days of rest

def weekday(day):
    return {0: 'MONDAY', 1: 'TUESDAY', 2: 'WEDNESDAY', 3: 'THURSDAY', 4: 'FRIDAY', 5: 'SATURDAY', 6: 'SUNDAY'}[day % 7]

def crange(start, end, modulo):
    if start > end:
        while start < modulo:
            yield start
            start += 1
        start = 0

    while start < end:
        yield start
        start += 1

# Linear Programming

In [926]:
from pulp import LpMinimize, LpMaximize, LpProblem, LpStatus, lpSum, LpVariable


# Model definition
model = LpProblem(name='exercise-scheduling', sense=LpMinimize)

# Initialize the decision variables
X = {}
Y = []

# X_i_j := Muscle group i is trained on the j-th day, i \in groups, j \in [0,14)
for group in groups:
    for day in days:
        X[group.name, day] = LpVariable(name='x_{}_{}'.format(group.name, day), cat='Binary')

# Y_j := the j-th day is a rest day, j \in [0, 14)
for day in days:
    Y.append(LpVariable(name='y_{}'.format(day), cat='Binary'))

# Z := the maximum number of muscle groups trained in any given day
# <=>
# Z := max_j { sum_i (X_i_j) }
Z = LpVariable(name='z', lowBound=0, upBound=len(groups), cat='Integer')


## Linear Constraints

In [927]:

# Each muscle group i must be hit exactly i.rotations times each microcycle 
for group in groups:
    model += sum( [ X[group.name, day] for day in days ] ) == group.rotations
    
# In each microcycle there have to be exactly `rest_days` rest days
model += sum(Y) == rest_days

# Max consecutive days of training/resting constraints
for day in days:
    maxwork = crange(day, (day + max_consecutive_work + 1) % len(days), len(days))
    maxrest = crange(day, (day + max_consecutive_rest + 1) % len(days), len(days))
    model += sum( [ (1 - Y[d]) for d in maxwork ] ) <= max_consecutive_work
    model += sum( [ Y[d] for d in maxrest ] ) <= max_consecutive_rest

# For each day j, if Y_j is set to 1 (j is a rest day), there must be no workouts scheduled for that day
# and if Y_j is set to 0, there has to be at least one muscle group scheduled for that day
for day in days:
    model += sum( [ X[group.name, day] for group in groups ] ) <= (1 - Y[day])*1000000
    model += sum( [ X[group.name, day] for group in groups ] ) >= (1 - Y[day])

# Threshold variable constraint (Z must represent the maximum number of muscle groups trained in any given day)
for day in days:
    model += sum( [ X[group.name, day] for group in groups ] ) <= Z
    
# Constraints determining the minimum and maximum number of days between two consecutive muscle group workouts
for group in groups:
    for day in days:
        maxdays = crange(day, (day + group.rest_max + 1) % len(days), len(days))
        mindays = crange(day, (day + group.rest_min + 1) % len(days), len(days))

        model += sum( [ X[group.name, d] for d in maxdays ] ) >= 1
        model += sum( [ X[group.name, d] for d in mindays ] ) <= 1
        
# Each muscle group i such that i.after_rest == True must always be trained after a rest day
for group in groups:
    for day in days:
        model += X[group.name, day] * group.after_rest <= Y[(day - 1) % len(days)]
        
# Muscle groups combination preferences:
# Each muscle group can only be trained with groups beloging to group.split_preference
for group in groups:
    # For every other group with which the first one CANNOT be trained
    for other in groups - group.split_preference - set([group]):
        for day in days:
            model += X[group.name, day] + X[other.name, day] <= 1

# Objective function
model += Z # we want to minimize the number of muscles trained per session


In [928]:
# Model definition summary
# model

## Model Output

In [929]:
# Solve the problem
status = model.solve() 
print('Status code (1 = Optimum found): {}, {}'.format(status, LpStatus[status]))

print('Objective (max groups per session): {}'.format(model.objective.value()))

# for var in model.variables():
#    print(f"{var.name}: {var.value()}")
    
print('SCHEDULE: ')
for day in days:
    split = [ (X[group.name, day], group.name) for group in groups ]
    split = list(filter(lambda g : g[0].value() == 1, split))
    split = list(map(lambda g: g[1], split))
    print('{}: {}'.format(weekday(day), split))


Status code (1 = Optimum found): 1, Optimal
Objective (max groups per session): 2.0
SCHEDULE: 
MONDAY: ['DELTS', 'BACK']
TUESDAY: ['CHEST']
WEDNESDAY: []
THURSDAY: ['ARMS']
FRIDAY: []
SATURDAY: ['DELTS', 'BACK']
SUNDAY: ['ARMS', 'CHEST']
MONDAY: []
TUESDAY: ['LEGS']
WEDNESDAY: ['ARMS', 'BACK']
THURSDAY: ['DELTS', 'CHEST']
FRIDAY: []
SATURDAY: []
SUNDAY: ['LEGS']
